In [11]:
import numpy as np
import pandas as pd

In [12]:
# traitement réalisés précédemment
prets = pd.read_csv('https://raw.githubusercontent.com/benjaminmrl/data-4452741/main/prets.csv')

# calcul du taux d'endettement
prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

# renommer taux en taux_interet
prets.rename(columns={'taux':'taux_interet'}, inplace=True)

# calculer le cout total du pret
prets['cout_total'] = prets['remboursement'] * prets['duree']

# calculer les bénéfices mensuels réalisés
prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

# création d'une variable risque
prets['risque'] = 'Non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'Oui'

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,Non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,Non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,Non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,Non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,Non


In [22]:
profil_clients = prets.groupby('identifiant')[['remboursement','taux_endettement','cout_total','benefices']].sum()
profil_clients.reset_index(inplace=True)
profil_clients.head()

,identifiant,remboursement,taux_endettement,cout_total,benefices
0,0,1130.05,30.80,271212.0,131.99
1,1,1487.85,28.02,389715.0,206.66
2,2,552.54,29.50,132609.6,53.71
3,3,586.03,34.80,105485.4,44.57
4,4,423.61,28.70,101666.4,51.21


In [24]:
# Recalculez le nombre exact de personnes en situation bancaire risquée à partir du taux d'endettemment :
client_risque = profil_clients.loc[profil_clients['taux_endettement'] > 35 , :].shape[0]

62

In [29]:
# Calculez le bénéfice dégagé par chacune de nos agences, par types de prêts. 
# Vous présenterez vos résultats sous la forme d'un tableau simple (via un group_by) :
benefice_par_agence_par_type = prets.groupby(['ville','type'])['benefices'].sum()
benefice_par_agence_par_type.head()

ville      type      
BORDEAUX   immobilier    1681.32
LYON       automobile      47.39
           immobilier    2967.41
MARSEILLE  automobile      61.36
           immobilier    4334.55
Name: benefices, dtype: float64

In [30]:
# Pour aller plus loin, on souhaite avoir un tableau à double entrée (via un pivot_table) présentant cette fois ci 
# les bénéfices moyen réalisés par chaque agence, pour chaque type de prêt :
prets.pivot_table(index = 'ville', columns = 'type', values = 'benefices', aggfunc = 'mean')

type,automobile,immobilier
ville,,
BORDEAUX,NaN,105.082500
LYON,15.796667,118.696400
MARSEILLE,20.453333,60.202083
NICE,15.575000,139.557895
PARIS,19.900000,137.734028
TOULOUSE,20.406667,86.535417
